## Transaction Model

### Importing Libraries

In [1]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np
import csv

Using TensorFlow backend.


### Input Text

In [2]:
#form input_texts
input_texts = []
with open('modified.csv') as csvfile:
    next(csvfile)
    readCSV = csv.reader(csvfile,delimiter=',')
    for row in readCSV:
        text = " ".join(row)
        input_texts.append(text)
input_texts[:5]

['1500 CAD Local currency monetary instrument withdrawals TRANSACTIONS EUR_50117251111835232242_T2 BIRD BB LIMITED 05-OCT-12 08.50.37.000000000 PM CH',
 '14200 CHF Local currency cash withdrawals TRANSACTIONS GBP_24465402366500770610_T2 CYBER WOOD LIMITED 11-OCT-12 08.12.55.000000000 AM CA',
 '980 CHF Travelers check purchase TRANSACTIONS GBP_13173081153909711647_T2 LOUIS GOODWIN TRUST 11-OCT-12 04.39.05.000000000 AM CH',
 '1870 CHF Local currency monetary instrument deposits TRANSACTIONS GBP_07267797247481992642_T2 LOUIS GOODWIN TRUST 12-OCT-12 08.15.39.000000000 PM CH',
 '10000 EUR Local currency monetary instrument deposits TRANSACTIONS EUR_26723846775234113487_T2 Name 14-OCT-12 01.18.54.000000000 PM CH']

### Output Text

In [3]:
with open('myfile.txt','r',encoding='utf-8') as f:
    lines =   f.read().split('\n') 
target_texts = []
for line in lines:
    line = 'START_ ' + line + ' _END'
    target_texts.append(line)
target_texts = target_texts[:4807]

target_texts[:5]

['START_ A 1500 CAD Local currency monetary instrument withdrawals transaction was done at account#EUR_50117251111835232242_T2 of tenant code#2000000  originated by BIRD BB LIMITED on 05-OCT-12 08.50.37.000000000 PM from country code CH. _END',
 'START_ A 14200 CHF Local currency cash withdrawals transaction was done at account#GBP_24465402366500770610_T2 originated by CYBER WOOD LIMITED on 11-OCT-12 08.12.55.000000000 AM from country code CA. _END',
 'START_ A 980 CHF Travelers check purchase transaction was done at account#GBP_13173081153909711647_T2 originated by LOUIS GOODWIN TRUST on 11-OCT-12 04.39.05.000000000 AM from country code CH. _END',
 'START_ A 1870 CHF Local currency monetary instrument deposits transaction was done at account#GBP_07267797247481992642_T2 originated by LOUIS GOODWIN TRUST on 12-OCT-12 08.15.39.000000000 PM from country code CH. _END',
 'START_ A 10000 EUR Local currency monetary instrument deposits transaction was done at account#EUR_26723846775234113487

### Data Processing

Forming Input and Output Word Dictionaries

In [4]:
input_words = set()
target_words = set()
for input_text,target_text in zip(input_texts,target_texts):
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)  

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4807
Number of unique input tokens: 2842
Number of unique output tokens: 2857
Max sequence length for inputs: 20
Max sequence length for outputs: 32


Processing the Input and Output dictionaries such that the model would understand it

In [6]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]
    for t, word in enumerate(target_text.split(' ')):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word] 
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Building the Encoder

In [7]:
embedding_size = 150
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(150, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

W1110 11:18:48.803533 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 11:18:48.851405 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1110 11:18:48.862376 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



### Building the Decoder

In [8]:
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(150, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 150)    426300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 150)    428550      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

### Training the model

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=250,
          validation_split=0.2)

W1110 11:18:57.595324 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1110 11:18:57.624248 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W1110 11:18:57.778113 18664 deprecation.py:323] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1110 11:18:59.109843 18664 deprecation_wrapper.py:119] From C:\Users\devendra\Anaconda3\envs\tf-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:986: 

Train on 3845 samples, validate on 962 samples
Epoch 1/250
3845/3845 [==============================] - 15s 4ms/step - loss: 3.6064 - acc: 0.1328 - val_loss: 2.8951 - val_acc: 0.1823
Epoch 2/250
3845/3845 [==============================] - 8s 2ms/step - loss: 2.5911 - acc: 0.2931 - val_loss: 2.1054 - val_acc: 0.4076
Epoch 3/250
3845/3845 [==============================] - 8s 2ms/step - loss: 1.7755 - acc: 0.4957 - val_loss: 1.3667 - val_acc: 0.5585
Epoch 4/250
3845/3845 [==============================] - 8s 2ms/step - loss: 1.1920 - acc: 0.5774 - val_loss: 0.9836 - val_acc: 0.6285
Epoch 5/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.9394 - acc: 0.6185 - val_loss: 0.8281 - val_acc: 0.6412
Epoch 6/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.8202 - acc: 0.6387 - val_loss: 0.7388 - val_acc: 0.6582
Epoch 7/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.7441 - acc: 0.6608 - val_loss: 0.6799 - val_acc: 0.6830
Epoch 8

Epoch 61/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.1734 - acc: 0.7948 - val_loss: 0.4188 - val_acc: 0.7740
Epoch 62/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.1690 - acc: 0.7959 - val_loss: 0.4146 - val_acc: 0.7746
Epoch 63/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.1635 - acc: 0.7971 - val_loss: 0.4152 - val_acc: 0.7738
Epoch 64/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.1586 - acc: 0.7985 - val_loss: 0.4137 - val_acc: 0.7748
Epoch 65/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.1538 - acc: 0.7994 - val_loss: 0.4192 - val_acc: 0.7740
Epoch 66/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.1487 - acc: 0.8008 - val_loss: 0.4193 - val_acc: 0.7744
Epoch 67/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.1451 - acc: 0.8010 - val_loss: 0.4096 - val_acc: 0.7743
Epoch 68/250
3845/3845 [========================

3845/3845 [==============================] - 7s 2ms/step - loss: 0.0069 - acc: 0.8290 - val_loss: 0.3690 - val_acc: 0.7871
Epoch 122/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0063 - acc: 0.8290 - val_loss: 0.3713 - val_acc: 0.7862
Epoch 123/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0057 - acc: 0.8291 - val_loss: 0.3699 - val_acc: 0.7869
Epoch 124/250
3845/3845 [==============================] - 8s 2ms/step - loss: 0.0057 - acc: 0.8290 - val_loss: 0.3711 - val_acc: 0.7863
Epoch 125/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0053 - acc: 0.8290 - val_loss: 0.3784 - val_acc: 0.7864
Epoch 126/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0048 - acc: 0.8290 - val_loss: 0.3796 - val_acc: 0.7853
Epoch 127/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0043 - acc: 0.8291 - val_loss: 0.3717 - val_acc: 0.7872
Epoch 128/250
3845/3845 [==============================

3845/3845 [==============================] - 8s 2ms/step - loss: 7.1464e-04 - acc: 0.8292 - val_loss: 0.3686 - val_acc: 0.7903
Epoch 181/250
3845/3845 [==============================] - 7s 2ms/step - loss: 8.2449e-04 - acc: 0.8291 - val_loss: 0.3688 - val_acc: 0.7894
Epoch 182/250
3845/3845 [==============================] - 8s 2ms/step - loss: 5.8298e-04 - acc: 0.8292 - val_loss: 0.3602 - val_acc: 0.7899
Epoch 183/250
3845/3845 [==============================] - 7s 2ms/step - loss: 0.0011 - acc: 0.8291 - val_loss: 0.3688 - val_acc: 0.7895
Epoch 184/250
3845/3845 [==============================] - 7s 2ms/step - loss: 5.6390e-04 - acc: 0.8292 - val_loss: 0.3598 - val_acc: 0.7900
Epoch 185/250
3845/3845 [==============================] - 8s 2ms/step - loss: 3.5987e-04 - acc: 0.8292 - val_loss: 0.3669 - val_acc: 0.7895
Epoch 186/250
3845/3845 [==============================] - 7s 2ms/step - loss: 2.9418e-04 - acc: 0.8292 - val_loss: 0.3634 - val_acc: 0.7901
Epoch 187/250
3845/3845 [======

3845/3845 [==============================] - 8s 2ms/step - loss: 3.4073e-04 - acc: 0.8292 - val_loss: 0.3604 - val_acc: 0.7923
Epoch 239/250
3845/3845 [==============================] - 8s 2ms/step - loss: 2.0521e-04 - acc: 0.8292 - val_loss: 0.3617 - val_acc: 0.7917
Epoch 240/250
3845/3845 [==============================] - 8s 2ms/step - loss: 1.4032e-04 - acc: 0.8292 - val_loss: 0.3601 - val_acc: 0.7917
Epoch 241/250
3845/3845 [==============================] - 8s 2ms/step - loss: 2.2654e-04 - acc: 0.8292 - val_loss: 0.3643 - val_acc: 0.7919
Epoch 242/250
3845/3845 [==============================] - 7s 2ms/step - loss: 1.9057e-04 - acc: 0.8292 - val_loss: 0.3605 - val_acc: 0.7921
Epoch 243/250
3845/3845 [==============================] - 8s 2ms/step - loss: 6.6029e-04 - acc: 0.8292 - val_loss: 0.3630 - val_acc: 0.7923
Epoch 244/250
3845/3845 [==============================] - 8s 2ms/step - loss: 1.6946e-04 - acc: 0.8292 - val_loss: 0.3616 - val_acc: 0.7925
Epoch 245/250
3845/3845 [==

### Forming models for testing

In [10]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Method to decode the Input

In [11]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 300):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Predictions

In [12]:
for seq_index in range(5):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['1500 CAD Local currency monetary instrument withdrawals TRANSACTIONS EUR_50117251111835232242_T2 BIRD BB LIMITED 05-OCT-12 08.50.37.000000000 PM CH']
Decoded sentence:  A 1500 CAD Local currency monetary instrument withdrawals transaction was done at account#EUR_50117251111835232242_T2 of tenant code#2000000  originated by BIRD BB LIMITED on 05-OCT-12 08.50.37.000000000 PM from country code CH. _END
-
Input sentence: ['14200 CHF Local currency cash withdrawals TRANSACTIONS GBP_24465402366500770610_T2 CYBER WOOD LIMITED 11-OCT-12 08.12.55.000000000 AM CA']
Decoded sentence:  A 14200 CHF Local currency cash withdrawals transaction was done at account#GBP_24465402366500770610_T2 originated by CYBER WOOD LIMITED on 11-OCT-12 08.12.55.000000000 AM from country code CA. _END
-
Input sentence: ['980 CHF Travelers check purchase TRANSACTIONS GBP_13173081153909711647_T2 LOUIS GOODWIN TRUST 11-OCT-12 04.39.05.000000000 AM CH']
Decoded sentence:  A 980 CHF Travelers check purc

## Transfer Model

### Input File

In [13]:
input_texts = []
with open('transfer_features.csv') as csvfile:
    next(csvfile)
    readCSV = csv.reader(csvfile,delimiter=',')
    for row in readCSV:
        text = " ".join(row)
        input_texts.append(text)
input_texts[:5]

['2000000.0 t2_145 16311.0 USD MXN 28-JUN-12 01.04.15.000000000 AM missing MXN Branch of Bank Of England Receive - Money Transfer - Incoming',
 '2000000.0 t2_889 16311.0 USD MXN 28-JUN-12 01.04.15.000000000 AM missing MXN Branch of Bank Of England Receive - Money Transfer - Outgoing',
 '2000000.0 t2_145 3189.0 USD USD 29-JUN-12 01.31.17.000000000 PM missing USD Branch of Bank of Martin Luther King Send - Money Order - Incoming',
 '2000000.0 t2_145 3189.0 USD USD 29-JUN-12 01.31.17.000000000 PM missing USD Branch of Bank of Martin Luther King Send - Express Payment - Incoming',
 '2000000.0 t2_117 3189.0 USD USD 29-JUN-12 01.31.17.000000000 PM missing USD Branch of Bank of Martin Luther King Send - Express Payment - Outgoing']

### Output File

In [20]:
with open('demofile2.txt','r',encoding='utf-8') as f:
    lines =   f.read().split('\n') 
target_texts = []
for line in lines:
    line = 'START_ ' + line + ' _END'
    target_texts.append(line)

target_texts[:5]

['START_ A 16311 USD Receive - Money Transfer - Incoming transaction was done at account#t2_145 of tenant code#2000000  originated by Branch of Bank Of England on 28-JUN-12 01.04.15.000000000 AM in MXN currency for country code  missing in MXN currency for Branch of BANK OF CHARLOTTE as benificiery. _END',
 'START_ A 16311 USD Receive - Money Transfer - Outgoing transaction was done at account#t2_889 of tenant code#2000000  originated by Branch of Bank Of England on 28-JUN-12 01.04.15.000000000 AM in MXN currency for country code  missing in MXN currency for Branch of BANK OF CHARLOTTE as benificiery. _END',
 'START_ A 3189 USD Send - Money Order - Incoming transaction was done at account#t2_145 of tenant code#2000000  originated by Branch of Bank of Martin Luther King on 29-JUN-12 01.31.17.000000000 PM in USD currency for country code  missing in USD currency for Branch of Bank of France as benificiery. _END',
 'START_ A 3189 USD Send - Express Payment - Incoming transaction was done 

###  Data Processing 

In [21]:
input_words = set()
target_words = set()
for input_text,target_text in zip(input_texts,target_texts):
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)  

In [22]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 5997
Number of unique input tokens: 5088
Number of unique output tokens: 6303
Max sequence length for inputs: 25
Max sequence length for outputs: 55


In [23]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]
    for t, word in enumerate(target_text.split(' ')):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word] 
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Building the Encoder  

In [24]:
embedding_size = 150
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(150, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### Decoder 

In [25]:
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(150, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model1 = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 150)    763200      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 150)    945450      input_8[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LS

### Training the model 

In [26]:
model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model1.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=250,
          validation_split=0.2)

Train on 4797 samples, validate on 1200 samples
Epoch 1/250
4797/4797 [==============================] - 35s 7ms/step - loss: 3.9405 - acc: 0.0805 - val_loss: 3.4801 - val_acc: 0.0833
Epoch 2/250
4797/4797 [==============================] - 38s 8ms/step - loss: 2.9731 - acc: 0.1871 - val_loss: 2.6366 - val_acc: 0.2870
Epoch 3/250
4797/4797 [==============================] - 33s 7ms/step - loss: 1.9610 - acc: 0.4972 - val_loss: 1.7025 - val_acc: 0.5900
Epoch 4/250
4797/4797 [==============================] - 30s 6ms/step - loss: 1.1834 - acc: 0.6515 - val_loss: 1.2366 - val_acc: 0.6340
Epoch 5/250
4797/4797 [==============================] - 28s 6ms/step - loss: 0.8614 - acc: 0.6850 - val_loss: 1.0521 - val_acc: 0.6602
Epoch 6/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.7328 - acc: 0.7026 - val_loss: 0.9673 - val_acc: 0.6723
Epoch 7/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.6692 - acc: 0.7131 - val_loss: 0.9222 - val_acc: 0.6872


4797/4797 [==============================] - 25s 5ms/step - loss: 0.1230 - acc: 0.8202 - val_loss: 0.8217 - val_acc: 0.7392
Epoch 61/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.1183 - acc: 0.8213 - val_loss: 0.8251 - val_acc: 0.7396
Epoch 62/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.1147 - acc: 0.8220 - val_loss: 0.8299 - val_acc: 0.7384
Epoch 63/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.1105 - acc: 0.8230 - val_loss: 0.8150 - val_acc: 0.7400
Epoch 64/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.1065 - acc: 0.8241 - val_loss: 0.8289 - val_acc: 0.7388
Epoch 65/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.1035 - acc: 0.8247 - val_loss: 0.8296 - val_acc: 0.7401
Epoch 66/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.0992 - acc: 0.8254 - val_loss: 0.8271 - val_acc: 0.7417
Epoch 67/250
4797/4797 [==============================

Epoch 120/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0124 - acc: 0.8435 - val_loss: 0.8563 - val_acc: 0.7464
Epoch 121/250
4797/4797 [==============================] - 26s 6ms/step - loss: 0.0110 - acc: 0.8437 - val_loss: 0.8684 - val_acc: 0.7452
Epoch 122/250
4797/4797 [==============================] - 26s 5ms/step - loss: 0.0107 - acc: 0.8438 - val_loss: 0.8600 - val_acc: 0.7473
Epoch 123/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0106 - acc: 0.8437 - val_loss: 0.8569 - val_acc: 0.7462
Epoch 124/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0101 - acc: 0.8437 - val_loss: 0.8770 - val_acc: 0.7451
Epoch 125/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0099 - acc: 0.8438 - val_loss: 0.8665 - val_acc: 0.7452
Epoch 126/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0093 - acc: 0.8439 - val_loss: 0.8674 - val_acc: 0.7450
Epoch 127/250
4797/4797 [=========

4797/4797 [==============================] - 25s 5ms/step - loss: 0.0022 - acc: 0.8444 - val_loss: 0.8971 - val_acc: 0.7438
Epoch 180/250
4797/4797 [==============================] - 25s 5ms/step - loss: 0.0018 - acc: 0.8445 - val_loss: 0.8719 - val_acc: 0.7466
Epoch 181/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0017 - acc: 0.8445 - val_loss: 0.8807 - val_acc: 0.7467
Epoch 182/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0016 - acc: 0.8445 - val_loss: 0.8744 - val_acc: 0.7470
Epoch 183/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0017 - acc: 0.8444 - val_loss: 0.8762 - val_acc: 0.7460
Epoch 184/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0017 - acc: 0.8445 - val_loss: 0.8794 - val_acc: 0.7461
Epoch 185/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0019 - acc: 0.8444 - val_loss: 0.8828 - val_acc: 0.7458
Epoch 186/250
4797/4797 [=======================

4797/4797 [==============================] - 25s 5ms/step - loss: 8.1411e-04 - acc: 0.8445 - val_loss: 0.8883 - val_acc: 0.7468
Epoch 239/250
4797/4797 [==============================] - 26s 5ms/step - loss: 8.9484e-04 - acc: 0.8445 - val_loss: 0.8987 - val_acc: 0.7458
Epoch 240/250
4797/4797 [==============================] - 27s 6ms/step - loss: 6.4772e-04 - acc: 0.8445 - val_loss: 0.8872 - val_acc: 0.7468
Epoch 241/250
4797/4797 [==============================] - 24s 5ms/step - loss: 9.6485e-04 - acc: 0.8445 - val_loss: 0.8887 - val_acc: 0.7472
Epoch 242/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0012 - acc: 0.8444 - val_loss: 0.9003 - val_acc: 0.7460
Epoch 243/250
4797/4797 [==============================] - 24s 5ms/step - loss: 0.0011 - acc: 0.8445 - val_loss: 0.8942 - val_acc: 0.7466
Epoch 244/250
4797/4797 [==============================] - 24s 5ms/step - loss: 7.3764e-04 - acc: 0.8445 - val_loss: 0.9009 - val_acc: 0.7458
Epoch 245/250
4797/4797 [===

### Forming Model for testing 

In [27]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Decoding Sequence function 

In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 300):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# 

In [29]:
for seq_index in range(5):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['2000000.0 t2_145 16311.0 USD MXN 28-JUN-12 01.04.15.000000000 AM missing MXN Branch of Bank Of England Receive - Money Transfer - Incoming']
Decoded sentence:  A 16311 USD Receive - Money Transfer - Incoming transaction was done at account#t2_145 of tenant code#2000000  originated by Branch of Bank Of England on 28-JUN-12 01.04.15.000000000 AM in MXN currency for country code  missing in MXN currency for Branch of BANK OF CHARLOTTE as benificiery. _END
-
Input sentence: ['2000000.0 t2_889 16311.0 USD MXN 28-JUN-12 01.04.15.000000000 AM missing MXN Branch of Bank Of England Receive - Money Transfer - Outgoing']
Decoded sentence:  A 16311 USD Receive - Money Transfer - Outgoing transaction was done at account#t2_889 of tenant code#2000000  originated by Branch of Bank Of England on 28-JUN-12 01.04.15.000000000 AM in MXN currency for country code  missing in MXN currency for Branch of BANK OF CHARLOTTE as benificiery. _END
-
Input sentence: ['2000000.0 t2_145 3189.0 US